<a href="https://colab.research.google.com/github/JakubChmielewskiRepo/wstep-do-uczenia-maszynowego-175ic/blob/master/lab10/realPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#!pip install aiohttp
import time
import nest_asyncio
import requests
import threading
import json
from urllib import request
import multiprocessing
import  aiohttp, asyncio
import concurrent.futures



In [6]:
#Synchronicznie
def download_site(url, session):
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration} seconds")

Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jython.org
Read 277 from http://olympus.realpython.org/dice
Read 10394 from https://www.jyth

In [11]:
#Z wątkami

thread_local = threading.local()

def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    with session.get(url) as response:
        session.get(url)


def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration_thread = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration_thread} seconds")

Downloaded 160 in 2.904771089553833 seconds


In [16]:
#Asyncio
async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(session.get(url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    nest_asyncio.apply()
    asyncio.run(download_all_sites(sites))
    duration_asyncio = time.time() - start_time
    print(f"Downloaded {len(sites)} sites in {duration_asyncio} seconds")

Downloaded 160 sites in 0.4126293659210205 seconds


In [17]:
#Multiprocessing

session = None


def set_global_session():
    global session
    if not session:
        session = requests.Session()


def download_site(url):
    with session.get(url) as response:
        name = multiprocessing.current_process().name
        session.get(url)


def download_all_sites(sites):
    with multiprocessing.Pool(initializer=set_global_session) as pool:
        pool.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration_multiproc = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration_multiproc} seconds")

Downloaded 160 in 7.134905576705933 seconds


In [19]:
#CPU-Bound Synchronous

def cpu_bound(number):
    return sum(i * i for i in range(number))


def find_sums(numbers):
    for number in numbers:
        cpu_bound(number)


if __name__ == "__main__":
    numbers = [5_000_000 + x for x in range(20)]

    start_time = time.time()
    find_sums(numbers)
    duration_cpu_synchronous = time.time() - start_time
    print(f"Duration {duration_cpu_synchronous} seconds")

Duration 9.38219165802002 seconds


In [20]:
#CPU-Bound multiprocessing
def cpu_bound(number):
    return sum(i * i for i in range(number))


def find_sums(numbers):
    with multiprocessing.Pool() as pool:
        pool.map(cpu_bound, numbers)


if __name__ == "__main__":
    numbers = [5_000_000 + x for x in range(20)]

    start_time = time.time()
    find_sums(numbers)
    duration_cpu_multiprocessing = time.time() - start_time
    print(f"Duration {duration_cpu_multiprocessing} seconds")

Duration 10.347258806228638 seconds


In [24]:
#Speed comparison
print(f"Synchronic: {duration} seconds")
print(f"Threading: {duration_thread} seconds")
print(f"Asyncio: {duration_asyncio} seconds")
print(f"Multiprocessing: {duration_multiproc} seconds")
print(f"CPU-Bound Synchronous {duration_cpu_synchronous} seconds")
print(f"CPU-Bound multiprocessing {duration_cpu_multiprocessing} seconds")

Synchronic: 8.690134286880493 seconds
Threading: 2.904771089553833 seconds
Asyncio: 0.4126293659210205 seconds
Multiprocessing: 7.134905576705933 seconds
CPU-Bound Synchronous 9.38219165802002 seconds
CPU-Bound multiprocessing 10.347258806228638 seconds
